In [1]:
import polars as pl

df = pl.read_parquet("./aristotle.parquet")

In [ ]:
%pip install pip==24.0
%pip install spacy grecy

/Users/pletcher/.pyenv/versions/3.12.7/lib/python3.12/pty.py:95: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

If you really know what your doing, you can silence this warning with the warning module
or by setting POLARS_ALLOW_FORKING_THREAD=1.

  pid, fd = os.forkpty()


  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: /Users/pletcher/code/writing/articles/2024-12-02_aristotle-methodos/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached spacy-3.8.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (27 kB)
  Using cached grecy-1.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.11-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.0 kB)
  Using cached cymem-2.0.10-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.4 kB)
  Using cached preshed-3.0.9-cp312-cp312-maco

In [2]:
import spacy

nlp = spacy.load("grc_proiel_lg")

In [ ]:
def get_lemmata(series: pl.Series):
    lemmata = []
    for s in series:
        doc = nlp(s)

        lemmata.append([t.lemma_ for t in doc if t.is_alpha])

    return pl.Series(lemmata)

out = df.select(pl.col("urn"), pl.col("unannotated_strings").map_batches(get_lemmata).alias("lemmata"))

In [10]:
from collections import Counter

lemmata_counts = Counter(out['lemmata'].explode())

In [44]:
total_non_stop_lemmata = len(out['lemmata'].explode())

print(len(lemmata_counts))

methodos = "μέθοδος"
arete = "ἀρετή"
philia = "φιλία"

methodos_ppm = (lemmata_counts[methodos] / total_non_stop_lemmata) * 1000000
arete_ppm = (lemmata_counts[arete] / total_non_stop_lemmata) * 1000000
philia_ppm = (lemmata_counts[philia] / total_non_stop_lemmata) * 1000000

print(f"methodos: {methodos_ppm} | arete: {arete_ppm} | philia: {philia_ppm}")


10450
methodos: 221.13533482491286 | arete: 4416.20271606223 | philia: 1814.6105416514906


In [ ]:
work_urns = out.with_columns(
    contains_methodos=pl.col('lemmata').list.contains(methodos),
    contains_arete=pl.col('lemmata').list.contains(arete),
    contains_philia=pl.col('lemmata').list.contains(philia),
)

work_urns.select(
    pl.col("contains_methodos").value_counts(),
    pl.col("contains_arete").value_counts(),
    pl.col("contains_philia").value_counts(),
)

contains_methodos,contains_arete,contains_philia
struct[2],struct[2],struct[2]
"{true,30}","{true,236}","{true,71}"
"{false,2474}","{false,2268}","{false,2433}"


In [42]:
import math

3 + math.log10(1814.6105416514906)

6.258783429503772